<a href="https://colab.research.google.com/github/andres-merino/Generador-Moodle-LaTeX/blob/main/Codigo%20Colab%20v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generador de preguntas de Moodle

In [21]:
#@title <font color="264CC7" size="+1"> Ejecutar configuración
# Paquetes
import itertools
import os
import random
import numpy as np
from scipy.stats import *
from sympy import *

from urllib.request import urlretrieve
urlretrieve("https://raw.githubusercontent.com/andres-merino/Generador-Moodle-LaTeX/main/Plantilla.tex", "/content/Plantilla.tex")
from google.colab import files
from IPython.display import HTML

## Parámetros y enunciado

Se deben completar los siguientes campos para la generación de las preguntas del cuestionario.

1. **Nombre del cuestionario:** Asigne un nombre descriptivo al cuestionario.
2. **Parámetros:** Enumere todos los parámetros utilizados en el la pregunta.
3. **Diccionario de parámetros:** Proporcione un diccionario con los parámetros y listas de valores de cada uno.
4. **Enunciado:** Escriba el enunciado de la pregunta. Utilice doble corchete `[[ ]]` para encerrar los parámetros o cálculos que desee que el generador reemplace.
    - Si un item dentro del enunciado no tiene opciones, use corchetes vacíos `[]`.
    - Incluya siempre el parámetro `[[id]]` en cada pregunta para su reemplazo automático.
5. **Cantidad máxima de ejercicios:** Especifique el número máximo de preguntas que desea que se generen para este cuestionario.

In [11]:
# Nombre del cuestionario
name = "Producto"

# Parámetros
par = ["a", "b"]

# Diccionario de parámetros
par_dict = {
    "a": [-2,-1,0,1,2],
    "b": [-2,2,3],
    }

# Enunciado
enunciado = r'''
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Producto - [[id]]}
    % - Enunciado
    Determine el valor de
    \[
        ([[a]])\cdot ([[b]])
    \]
    \item[] [[a * b]]
\end{numerical}
'''

# Cantidad máxima de ejercicios
max_ejercicios = 3


## Generar preguntas

Ejecutar la siguiente celda, se obtendrá el código en LaTeX de las perguntas generadas.

In [12]:
#@title <font color="264CC7" size="+1"> Generar preguntas
# Total preguntas
_total = max_ejercicios
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(10)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
_par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('% Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

# Mensaje
print(_quiz)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Producto - 1}
    % - Enunciado
    Determine el valor de
    \[
        (1)\cdot (-2)
    \]
    \item[] -2
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Producto - 2}
    % - Enunciado
    Determine el valor de
    \[
        (-2)\cdot (-2)
    \]
    \item[] 4
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Producto - 3}
    % - Enunciado
    Determine el valor de
    \[
        (0)\cdot (-2)
    \]
    \item[] 0
\end{numerical}




## Generar .tex

Al ejecutar la siguiente celda, se obtendrá el archivo .tex con el cuestionario.

In [25]:
#@title <font color="264CC7" size="+1"> Generar .tex
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))

# Semilla
random.seed(10)

# Si está definido el máximo de ejercicios
if max_ejercicios:
    # Tomo combinaciones aleatorias
    _par_comb = random.sample(_par_comb, max_ejercicios)

# Genero el examen
_quiz = ''
id = 1
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)

_parametros_latex = ''
for _parametro in par:
    _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\{' + ', '.join(map(str, par_dict[_parametro])) + '\}$' + '\n'

# Leo el archivo plantilla
_plantilla = open('Plantilla.tex', 'r', encoding='utf-8')
_t_plantilla = _plantilla.read()
_plantilla.close()
# Reemplazo
_t_plantilla = _t_plantilla.replace('{{QUIZ}}', _quiz)
_t_plantilla = _t_plantilla.replace('{{Cuestionario}}', name)
_t_plantilla = _t_plantilla.replace('{{Enunciado}}', enunciado)
_t_plantilla = _t_plantilla.replace('{{Parámetros}}', _parametros_latex)
_t_plantilla = _t_plantilla.replace('{{Número de preguntas}}', str(id-1))

# Escribo el archivo
_examen = open(f'{name}.tex', 'w', encoding='utf-8')
_examen.write(_t_plantilla)
_examen.close()

# Mensaje
print(f'Examen generado con éxito, se generaron {id-1} preguntas.')

# Descarga
files.download(f'{name}.tex')
print(f'Descargando archivo.')

Examen generado con éxito, se generaron 3 preguntas.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El archivo se ha descargado.
